In [4]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import sys
import pandas as pd
import numpy as np
import csv
import random
import re
from sklearn.decomposition import TruncatedSVD
from torch.nn.utils.rnn import pad_sequence#用来填充序列
import time
from prefetch_generator import BackgroundGenerator
from torch.utils.tensorboard import SummaryWriter
from pywick.optimizers.nadam import Nadam#使用pywick包里的nadam优化器
from torch.optim import lr_scheduler

In [2]:
train_path = ['/home/jsy/train_20160701-20170531','/media/jsy/Samsung/train_20180211-20190224','/media/jsy/WD/train_20170601-20180210']

In [5]:
filepath=train_path
lablelist = pd.read_csv('/home/jsy/data/lablelist.csv',index_col = 0)#比赛id及其对应赛果的列表
filelist0 = list()
for x in filepath:#把filepath这个列表中的所有目录下的路径组合成一个列表
    filelist0+=[i+'/'+k for i,j,k in os.walk(x) for k in k]#得到所有csv文件的路径列表
#self.filelist0 = [i+'/'+k for i,j,k in os.walk(filepath) for k in k]#得到所有csv文件的路径列表
filelist = [data_path for data_path in filelist0 if int(re.findall(r'/(\d*?).csv',data_path)[0]) in  lablelist.index]#只保留有赛果的文件路径
lables = {'win':0,'draw':1,'lose':2}#分类问题要从0开始编号,而且要对应好了表中的顺序编，


In [11]:
with open(file, 'rb') as f:  # 打开文件

    # 在文本文件中，没有使用b模式选项打开的文件，只允许从文件头开始,只能seek(offset,0)

    offset = -50  # 设置偏移量

    while True:

        """

        file.seek(off, whence=0)：从文件中移动off个操作标记（文件指针），正往结束方向移动，负往开始方向移动。

        如果设定了whence参数，就以whence设定的起始位为准，0代表从头开始，1代表当前位置，2代表文件最末尾位置。  

        """

        f.seek(offset, 2)  # seek(offset, 2)表示文件指针：从文件末尾(2)开始向前50个字符(-50)

        lines = f.readlines()  # 读取文件指针范围内所有行

        if len(lines) >= 2:  # 判断是否最后至少有两行，这样保证了最后一行是完整的

            last_line = lines[-1]  # 取最后一行

            break

        # 如果off为50时得到的readlines只有一行内容，那么不能保证最后一行是完整的

        # 所以off翻倍重新运行，直到readlines不止一行

        offset *= 2



In [34]:
max_list=[]
min_list=[]
mid_list=[]

In [36]:
def classify(data_path):
    with open(data_path, 'rb') as f:  # 打开文件

    # 在文本文件中，没有使用b模式选项打开的文件，只允许从文件头开始,只能seek(offset,0)

        offset = -50  # 设置偏移量

        while True:

            """

            file.seek(off, whence=0)：从文件中移动off个操作标记（文件指针），正往结束方向移动，负往开始方向移动。

            如果设定了whence参数，就以whence设定的起始位为准，0代表从头开始，1代表当前位置，2代表文件最末尾位置。  

            """

            f.seek(offset, 2)  # seek(offset, 2)表示文件指针：从文件末尾(2)开始向前50个字符(-50)

            lines = f.readlines()  # 读取文件指针范围内所有行

            if len(lines) >= 2:  # 判断是否最后至少有两行，这样保证了最后一行是完整的

                last_line = lines[-1]  # 取最后一行

                break

            # 如果off为50时得到的readlines只有一行内容，那么不能保证最后一行是完整的

            # 所以off翻倍重新运行，直到readlines不止一行

            offset *= 2
    last_line_list=last_line.decode().split(',')#把字符串转成列表，分隔符为逗号
    peilv =[float(last_line_list[7]),float(last_line_list[8]),float(last_line_list[9])]#把三种赔率装起来
    bisai_id = int(re.findall(r'/(\d*?).csv',data_path)[0])
    lable = lablelist.loc[bisai_id].result
    lable = lables[lable]#找到对应场比赛的结果
    result = peilv[lable]
    if result == min(peilv):#如果赢了的那个是赔率最小的那个
        min_list.append(data_path)
    elif result == max(peilv):
        max_list.append(data_path)
    else:
        mid_list.append(data_path)
        
        
    


    
    

In [37]:
list(map(classify,filelist))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [39]:
len(max_list)

13141

In [40]:
len(mid_list)

14838

In [41]:
len(min_list)

31397

In [54]:
a = min_list[0:13141]

In [47]:
min_list[0]

'/home/jsy/train_20160701-20170531/2016-12-30/880528.csv'

In [57]:
a[0]

'/home/jsy/train_20160701-20170531/2016-12-30/880528.csv'

In [55]:
b = mid_list[0:13141]

In [52]:
mid_list[0]

'/home/jsy/train_20160701-20170531/2016-12-30/904851.csv'

In [58]:
b[0]

'/home/jsy/train_20160701-20170531/2016-12-30/904851.csv'

In [59]:
balanced_train_path = max_list+a+b

In [60]:
len(balanced_train_path)

39423

In [61]:
with open('./balanced_train_path.txt', 'w') as f:#写入txt在notebooks文件夹下
    for item in balanced_train_path:
        f.write("%s\n" % item)

In [62]:
test_path = ['/media/jsy/Samsung/test2','/home/jsy/test4']
filepath=test_path
lablelist = pd.read_csv('/home/jsy/data/lablelist.csv',index_col = 0)#比赛id及其对应赛果的列表
filelist0 = list()
for x in filepath:#把filepath这个列表中的所有目录下的路径组合成一个列表
    filelist0+=[i+'/'+k for i,j,k in os.walk(x) for k in k]#得到所有csv文件的路径列表
#self.filelist0 = [i+'/'+k for i,j,k in os.walk(filepath) for k in k]#得到所有csv文件的路径列表
filelist = [data_path for data_path in filelist0 if int(re.findall(r'/(\d*?).csv',data_path)[0]) in  lablelist.index]#只保留有赛果的文件路径
lables = {'win':0,'draw':1,'lose':2}#分类问题要从0开始编号,而且要对应好了表中的顺序编，
max_list=[]
min_list=[]
mid_list=[]       
def classify(data_path):
    with open(data_path, 'rb') as f:  # 打开文件

    # 在文本文件中，没有使用b模式选项打开的文件，只允许从文件头开始,只能seek(offset,0)

        offset = -50  # 设置偏移量

        while True:

            """

            file.seek(off, whence=0)：从文件中移动off个操作标记（文件指针），正往结束方向移动，负往开始方向移动。

            如果设定了whence参数，就以whence设定的起始位为准，0代表从头开始，1代表当前位置，2代表文件最末尾位置。  

            """

            f.seek(offset, 2)  # seek(offset, 2)表示文件指针：从文件末尾(2)开始向前50个字符(-50)

            lines = f.readlines()  # 读取文件指针范围内所有行

            if len(lines) >= 2:  # 判断是否最后至少有两行，这样保证了最后一行是完整的

                last_line = lines[-1]  # 取最后一行

                break

            # 如果off为50时得到的readlines只有一行内容，那么不能保证最后一行是完整的

            # 所以off翻倍重新运行，直到readlines不止一行

            offset *= 2
    last_line_list=last_line.decode().split(',')#把字符串转成列表，分隔符为逗号
    peilv =[float(last_line_list[7]),float(last_line_list[8]),float(last_line_list[9])]#把三种赔率装起来
    bisai_id = int(re.findall(r'/(\d*?).csv',data_path)[0])
    lable = lablelist.loc[bisai_id].result
    lable = lables[lable]#找到对应场比赛的结果
    result = peilv[lable]
    if result == min(peilv):#如果赢了的那个是赔率最小的那个
        min_list.append(data_path)
    elif result == max(peilv):
        max_list.append(data_path)
    else:
        mid_list.append(data_path)
        

list(map(classify,filelist))


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [63]:
len(max_list)

3005

In [64]:
len(min_list)

6914

In [65]:
len(mid_list)

3240

In [66]:
len_max = len(max_list)
balanced_test_path = max_list+min_list[0:len_max]+mid_list[0:len_max]

In [67]:
len(balanced_test_path)

9015

In [68]:
with open('./balanced_test_path.txt', 'w') as f:#写入txt在notebooks文件夹下
    for item in balanced_test_path:
        f.write("%s\n" % item)